# Importing Stuffs

In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_profiling
import missingno as mso
import seaborn as sns
import warnings
import os
import yellowbrick
import scipy.cluster.hierarchy as shc

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
df = pd.read_csv('/content/drive/MyDrive/wfp_food_prices_mwi.csv')
df = df[1:]
df.head()

,date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice
1,1990-10-15,NaN,NaN,National Average,NaN,NaN,non-food,Fuel (diesel),L,actual,Retail,MWK,1.9,0.0019
2,1990-10-15,NaN,NaN,National Average,NaN,NaN,non-food,"Fuel (kerosene, paraffin)",L,actual,Retail,MWK,1.22,0.0012
3,1991-08-15,NaN,NaN,National Average,NaN,NaN,non-food,Fuel (diesel),L,actual,Retail,MWK,2.0,0.0019
4,1991-08-15,NaN,NaN,National Average,NaN,NaN,non-food,"Fuel (kerosene, paraffin)",L,actual,Retail,MWK,1.22,0.0012
5,1992-06-15,NaN,NaN,National Average,NaN,NaN,non-food,Fuel (diesel),L,actual,Retail,MWK,2.58,0.0025


In [47]:
df.price = pd.to_numeric(df.price)
df.usdprice = pd.to_numeric(df.usdprice)
df.date = pd.to_datetime(df.date)
df.head()

,date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice
1,1990-10-15,NaN,NaN,National Average,NaN,NaN,non-food,Fuel (diesel),L,actual,Retail,MWK,1.90,0.0019
2,1990-10-15,NaN,NaN,National Average,NaN,NaN,non-food,"Fuel (kerosene, paraffin)",L,actual,Retail,MWK,1.22,0.0012
3,1991-08-15,NaN,NaN,National Average,NaN,NaN,non-food,Fuel (diesel),L,actual,Retail,MWK,2.00,0.0019
4,1991-08-15,NaN,NaN,National Average,NaN,NaN,non-food,"Fuel (kerosene, paraffin)",L,actual,Retail,MWK,1.22,0.0012
5,1992-06-15,NaN,NaN,National Average,NaN,NaN,non-food,Fuel (diesel),L,actual,Retail,MWK,2.58,0.0025


# Food Price

In [48]:
df2 = df[['date', 'admin1', 'admin2', 'commodity', 'price', 'usdprice']]

In [49]:
df2.commodity.value_counts()

Maize                        12211
Beans                         3392
Rice                          3194
Groundnuts (shelled)          2577
Cassava                       1256
Cowpeas                        476
Maize (white)                  250
Rice (imported)                248
Sorghum (red)                  219
Pigeon peas                    169
Fuel (diesel)                  165
Fuel (kerosene, paraffin)      165
Fuel (petrol)                  159
Name: commodity, dtype: int64

In [50]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24481 entries, 1 to 24481
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       24481 non-null  datetime64[ns]
 1   admin1     23992 non-null  object        
 2   admin2     23992 non-null  object        
 3   commodity  24481 non-null  object        
 4   price      24481 non-null  float64       
 5   usdprice   24481 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 1.1+ MB


In [51]:
dataset_cities = df2.admin2.unique()
dataset_cities

array([nan, 'Nsanje', 'Dowa', 'Lilongwe', 'Mchinji', 'Nkhotakota',
       'Ntcheu', 'Ntchisi', 'Chitipa', 'Karonga', 'Mzimba', 'Rumphi',
       'Chikwawa', 'Machinga', 'Mangochi', 'Thyolo', 'Salima',
       'Nkhata Bay', 'Kasungu', 'Blantyre', 'Dedza', 'Balaka',
       'Lilongwe City', 'Chiradzulu', 'Mwanza', 'Blantyre City',
       'Zomba City', 'Mulanje', 'Neno', 'Zomba', 'Phalombe', 'Mzuzu City'],
      dtype=object)

In [52]:
dataset_cities.shape[0] - 2

30

In [53]:
survey_cities = ['Chitipa', 'Karonga', 'Nkhatabay', 'Rumphi', 'Mzimba', 'Mzuzu City', 'Kasungu', 
  'Nkhota kota', 'Ntchisi', 'Dowa', 'Salima', 'Lilongwe', 'Mchinji', 'Dedza', 'Ntcheu',
  'Lilongwe City', 'Mangochi', 'Machinga', 'Zomba', 'Chiradzulu', 'Blanytyre', 'Mwanza', 
  'Thyolo', 'Mulanje', 'Phalombe', 'Chikwawa', 'Nsanje', 'Balaka', 'Neno', 'Zomba City',
  'Blantyre City']

len(survey_cities)

31

In [54]:
for city in survey_cities:
    if city not in dataset_cities:
        print(city)

Nkhatabay
Nkhota kota
Blanytyre


## City (admin2)

In [55]:
comm_list = ['Maize', 'Rice', 'Cassava', 'Beans', 'Groundnuts (shelled)']
city_list = ['Chitipa', 'Karonga', 'Nkhata Bay', 'Rumphi', 'Mzimba', 'Mzuzu City', 'Kasungu', 'Nkhotakota', 'Ntchisi', 'Dowa', 'Salima', 'Lilongwe', 'Mchinji', 'Dedza', 'Ntcheu', 'Lilongwe City', 'Mangochi', 'Machinga', 'Zomba', 'Chiradzulu', 'Blantyre', 'Mwanza', 'Thyolo', 'Mulanje', 'Phalombe', 'Chikwawa', 'Nsanje', 'Balaka', 'Neno', 'Zomba City', 'Blantyre City']

In [56]:
def moving_average(start_date, end_date, csv_name):
    
    df_temp = pd.DataFrame()
    df_final = pd.DataFrame()
    
    for x in range(len(comm_list)):
        for i in range(len(city_list)):
            df_1 = df2[df2.commodity == comm_list[x]]
            df_1 = df_1[df_1.admin2 == city_list[i]]
            df_1 = df_1[(df_1['date'] > start_date) & (df_1['date'] < end_date)]

            df_1_x = df_1.groupby(['date'])[['price', 'usdprice']].transform('sum').drop_duplicates()
            df_1_x['date'] = df_1.date
            df_1_x['count'] = df_1.groupby(['date'])['date'].transform('count')
            df_1_x['price_avg'] = df_1_x['price']/df_1_x['count']
            df_1_x['usdprice_avg'] = df_1_x['usdprice']/df_1_x['count']

            df_1_x['commodity'] = comm_list[x]
            df_1_x['city'] = city_list[i]

            df_temp = pd.concat([df_temp, df_1_x], axis=0)
            
    df_temp = df_temp[['date', 'commodity', 'city', 'count', 'price', 'price_avg', 'usdprice', 'usdprice_avg']].reset_index()

    del df_temp['index']
    # df_temp.to_csv(f'df_temp_{csv_name}.csv')

    df_final = df_temp.groupby(['commodity', 'city'])[['price_avg', 'usdprice_avg']].mean()
    df_final['start_date'] = df_temp.groupby(['commodity', 'city'])[['date']].min()
    df_final['end_date'] =  df_temp.groupby(['commodity', 'city'])[['date']].max()
    df_final['date_count'] = df_temp.groupby(['commodity', 'city'])[['date']].count()
    # df_final.to_csv(f'city_{csv_name}.csv')
    return df_final

In [57]:
c3m19 = moving_average('2019-08-01', '2019-10-31', 'food_3m_2019')
c6m19 = moving_average('2019-05-01', '2019-10-31', 'food_6m_2019')
c12m19 = moving_average('2018-11-01', '2019-10-31', 'food_12m_2019')

c3m16 = moving_average('2016-08-01', '2016-10-31', 'food_3m_2016')
c6m16 = moving_average('2016-05-01', '2016-10-31', 'food_6m_2016')
c12m16 = moving_average('2015-11-01', '2016-10-31', 'food_12m_2016')

c3m13 = moving_average('2013-05-01', '2013-07-30', 'food_3m_2013')
c6m13 = moving_average('2013-02-01', '2013-07-30', 'food_6m_2013')
c12m13 = moving_average('2012-08-01', '2013-07-30', 'food_12m_2013')

In [58]:
def merge_df(df_list, period):
    new_df = pd.DataFrame()
    for x in range(len(df_list)):
        new_df = pd.concat([new_df, df_list[x]])
    new_df['period'] = period
    new_df = new_df.reset_index()
    new_df = new_df[new_df['commodity'] != "Cassava"].reset_index(drop=True)
    return new_df

#### Filter MA based on date count

In [59]:
df_list = [c3m13, c3m16, c3m19]
period = '3'
city_3m = merge_df(df_list, period)
(city_3m.date_count < 2).sum()

37

In [60]:
city_3m_2 = city_3m[city_3m.date_count >= 2]

In [61]:
df_list = [c6m13, c6m16, c6m19]
period = '6'
city_6m = merge_df(df_list, period)
(city_6m.date_count < 4).sum()

91

In [62]:
city_6m_2 = city_6m[city_6m.date_count >= 4]

In [63]:
df_list = [c12m13, c12m16, c12m19]
period = '12'
city_12m = merge_df(df_list, period)
(city_12m.date_count < 7).sum()

75

In [64]:
city_12m_2 = city_12m[city_12m.date_count >= 7]

#### 2013

In [65]:
# city_3m_13 = city_3m_2[city_3m_2.end_date.dt.year == 2013][['commodity', 'city', 'usdprice_avg']]
# city_3m_13 = city_3m_13.set_index(['commodity', 'city'])
# city_3m_13.columns = ["ma_3_usd"]
# city_6m_13 = city_6m_2[city_6m_2.end_date.dt.year == 2013][['commodity', 'city', 'usdprice_avg']]
# city_6m_13 = city_6m_13.set_index(['commodity', 'city'])
# city_6m_13.columns = ["ma_6_usd"]
# city_12m_13 = city_12m_2[city_12m_2.end_date.dt.year == 2013][['commodity', 'city', 'usdprice_avg']]
# city_12m_13 = city_12m_13.set_index(['commodity', 'city'])
# city_12m_13.columns = ["ma_12_usd"]

In [66]:
def merge_ma(year):
    city_3m_y = city_3m_2[city_3m_2.end_date.dt.year == year][['commodity', 'city', 'usdprice_avg']]
    city_3m_y = city_3m_y.set_index(['commodity', 'city'])
    city_3m_y.columns = ["ma_3_usd"]
    city_6m_y = city_6m_2[city_6m_2.end_date.dt.year == year][['commodity', 'city', 'usdprice_avg']]
    city_6m_y = city_6m_y.set_index(['commodity', 'city'])
    city_6m_y.columns = ["ma_6_usd"]
    city_12m_y = city_12m_2[city_12m_2.end_date.dt.year == year][['commodity', 'city', 'usdprice_avg']]
    city_12m_y = city_12m_y.set_index(['commodity', 'city'])
    city_12m_y.columns = ["ma_12_usd"]
    merged_ma = city_3m_y.merge(city_6m_y, left_index=True, right_index=True)
    merged_ma = merged_ma.merge(city_12m_y, left_index=True, right_index=True)
    return merged_ma

In [67]:
city_13 = merge_ma(2013)

In [68]:
city_13.to_csv("city_food_price_13.csv")

#### 2016

In [69]:
city_16 = merge_ma(2016)

In [70]:
city_16.to_csv("city_food_price_16.csv")

#### 2019

In [71]:
city_19 = merge_ma(2019)

In [72]:
city_19.to_csv("city_food_price_19.csv")

## Region (admin1)

In [73]:
comm_list = ['Maize', 'Rice', 'Cassava', 'Beans', 'Groundnuts (shelled)']
region_list = ['Southern Region', 'Northern Region', 'Central Region']

In [74]:
def moving_average(start_date, end_date, csv_name):
    
    df_temp = pd.DataFrame()
    df_final = pd.DataFrame()
    
    for x in range(len(comm_list)):
        for i in range(len(region_list)):
            df_1 = df2[df2.commodity == comm_list[x]]
            df_1 = df_1[df_1.admin1 == region_list[i]]
            df_1 = df_1[(df_1['date'] > start_date) & (df_1['date'] < end_date)]
            
            df_1_x = df_1.groupby(['date'])[['price', 'usdprice']].transform('sum').drop_duplicates()
            df_1_x['date'] = df_1.date
            df_1_x['count'] = df_1.groupby(['date'])['date'].transform('count')
            df_1_x['price_avg'] = df_1_x['price']/df_1_x['count']
            df_1_x['usdprice_avg'] = df_1_x['usdprice']/df_1_x['count']
            
            df_1_x['commodity'] = comm_list[x]
            df_1_x['region'] = region_list[i]

            df_temp = pd.concat([df_temp, df_1_x], axis=0)
            
    df_temp = df_temp[['date', 'commodity', 'region', 'count', 'price', 'price_avg', 'usdprice', 'usdprice_avg']].reset_index()

    del df_temp['index']
    # df_temp.to_csv(f'df_temp_{csv_name}.csv')

    df_final = df_temp.groupby(['commodity', 'region'])[['price_avg', 'usdprice_avg']].mean()
    df_final['start_date'] = df_temp.groupby(['commodity', 'region'])[['date']].min()
    df_final['end_date'] =  df_temp.groupby(['commodity', 'region'])[['date']].max()
    df_final['date_count'] = df_temp.groupby(['commodity', 'region'])[['date']].count()
    # df_final.to_csv(f'region_{csv_name}.csv')
    return df_final

In [75]:
r3m19 = moving_average('2019-08-01', '2019-10-31', 'food_3m_2019')
r6m19 = moving_average('2019-05-01', '2019-10-31', 'food_6m_2019')
r12m19 = moving_average('2018-11-01', '2019-10-31', 'food_12m_2019')

r3m16 = moving_average('2016-08-01', '2016-10-31', 'food_3m_2016')
r6m16 = moving_average('2016-05-01', '2016-10-31', 'food_6m_2016')
r12m16 = moving_average('2015-11-01', '2016-10-31', 'food_12m_2016')

r3m13 = moving_average('2013-05-01', '2013-07-30', 'food_3m_2013')
r6m13 = moving_average('2013-02-01', '2013-07-30', 'food_6m_2013')
r12m13 = moving_average('2012-08-01', '2013-07-30', 'food_12m_2013')

In [76]:
df_list = [r3m13, r3m16, r3m19]
period = '3'
region_3m = merge_df(df_list, period)
(region_3m.date_count < 2).sum()

0

In [77]:
region_3m.shape

(36, 8)

In [78]:
df_list = [r6m13, r6m16, r6m19]
period = '6'
region_6m = merge_df(df_list, period)
(region_6m.date_count < 4).sum()

0

In [79]:
region_6m.shape

(36, 8)

In [80]:
df_list = [r12m13, r12m16, r12m19]
period = '12'
region_12m = merge_df(df_list, period)
(region_12m.date_count < 7).sum()

0

In [81]:
region_12m.shape

(36, 8)

In [82]:
regional_price = pd.concat([region_3m, region_6m])
regional_price = pd.concat([regional_price, region_12m]).reset_index(drop=True)
replacer = {"Central Region":"Central", "Southern Region":"Southern", "Northern Region":"Northern"}
regional_price['region'] = regional_price['region'].replace(replacer)
regional_price.to_csv('regional_food_price.csv')

In [89]:
regional_price.head()

,commodity,region,price_avg,usdprice_avg,start_date,end_date,date_count,period
0,Beans,Central,337.102749,0.328274,2013-05-15,2013-07-15,3,3
1,Beans,Northern,314.950038,0.306703,2013-05-15,2013-07-15,3,3
2,Beans,Southern,364.872792,0.355310,2013-05-15,2013-07-15,3,3
3,Groundnuts (shelled),Central,350.198018,0.341014,2013-05-15,2013-07-15,3,3
4,Groundnuts (shelled),Northern,345.230869,0.336185,2013-05-15,2013-07-15,3,3


# Fuel Price

## National 

In [83]:
df3 = df[['date', 'market', 'commodity', 'price', 'usdprice']]

In [84]:
comm_list2 = ['Fuel (diesel)', 'Fuel (kerosene, paraffin)', 'Fuel (petrol)']

In [85]:
def moving_average2(start_date, end_date, csv_name):
    
    df_temp = pd.DataFrame()
    df_final = pd.DataFrame()
    
    for x in range(len(comm_list2)):
        df_1 = df3[df3.commodity == comm_list2[x]]
        
        df_1 = df_1[(df_1['date'] > start_date) & (df_1['date'] < end_date)]
        
        df_1_x = df_1.groupby(['date']).transform('sum')
        df_1_x = df_1_x.merge(df_1.date, left_index=True, right_index=True)
        df_1_x['market_count'] = df_1.groupby(['date'])['date'].transform('count')
        df_1_x['price_avg'] = df_1_x['price']/df_1_x['market_count']
        df_1_x['usdprice_avg'] = df_1_x['usdprice']/df_1_x['market_count']
                    
        df_1_x['commodity'] = comm_list2[x]
        df_1_x['market'] = "National Average"
        df_temp = pd.concat([df_temp, df_1_x], axis=0)
        

    df_temp = df_temp[['date','commodity', 'market', 'market_count', 'price', 'price_avg', 'usdprice', 'usdprice_avg']].reset_index()

    df_temp.to_csv(f'df_temp_{csv_name}.csv')
    df_final = df_temp.groupby(['commodity', 'market'])[['price_avg', 'usdprice_avg']].mean()
    df_final['start_date'] = df_temp.groupby(['commodity', 'market'])[['date']].min()
    df_final['end_date'] =  df_temp.groupby(['commodity', 'market'])[['date']].max()
    df_final['date_count'] = df_temp.groupby(['commodity', 'market'])[['date']].count()
    display(df_final)
    df_final.to_csv(f'df_final_{csv_name}.csv')

In [86]:
moving_average2('2019-08-01', '2019-10-31', '3m_2019')
moving_average2('2019-05-01', '2019-10-31', '6m_2019')
moving_average2('2018-11-01', '2019-10-31', '12m_2019')

moving_average2('2016-08-01', '2016-10-31', '3m_2016')
moving_average2('2016-05-01', '2016-10-31', '6m_2016')
moving_average2('2015-11-01', '2016-10-31', '12m_2016')

moving_average2('2013-05-01', '2013-07-30', '3m_2013')
moving_average2('2013-02-01', '2013-07-30', '6m_2013')
moving_average2('2012-07-01', '2013-07-30', '12m_2013')

,,price_avg,usdprice_avg,start_date,end_date,date_count
commodity,market,,,,,
Fuel (diesel),National Average,874.0,0.8511,2019-08-15,2019-10-15,3
"Fuel (kerosene, paraffin)",National Average,710.5,0.6919,2019-08-15,2019-10-15,3
Fuel (petrol),National Average,868.0,0.8453,2019-08-15,2019-10-15,3


,,price_avg,usdprice_avg,start_date,end_date,date_count
commodity,market,,,,,
Fuel (diesel),National Average,874.0,0.8511,2019-05-15,2019-10-15,6
"Fuel (kerosene, paraffin)",National Average,710.5,0.6919,2019-05-15,2019-10-15,6
Fuel (petrol),National Average,868.0,0.8453,2019-05-15,2019-10-15,6


,,price_avg,usdprice_avg,start_date,end_date,date_count
commodity,market,,,,,
Fuel (diesel),National Average,890.000000,0.866683,2018-11-15,2019-10-15,12
"Fuel (kerosene, paraffin)",National Average,722.650000,0.703725,2018-11-15,2019-10-15,12
Fuel (petrol),National Average,882.816667,0.859725,2018-11-15,2019-10-15,12


,,price_avg,usdprice_avg,start_date,end_date,date_count
commodity,market,,,,,
Fuel (diesel),National Average,766.9,0.7468,2016-08-15,2016-10-15,3
"Fuel (kerosene, paraffin)",National Average,609.8,0.5938,2016-08-15,2016-10-15,3
Fuel (petrol),National Average,788.3,0.7677,2016-08-15,2016-10-15,3


,,price_avg,usdprice_avg,start_date,end_date,date_count
commodity,market,,,,,
Fuel (diesel),National Average,759.55,0.739650,2016-05-15,2016-10-15,6
"Fuel (kerosene, paraffin)",National Average,604.90,0.589033,2016-05-15,2016-10-15,6
Fuel (petrol),National Average,780.80,0.760383,2016-05-15,2016-10-15,6


,,price_avg,usdprice_avg,start_date,end_date,date_count
commodity,market,,,,,
Fuel (diesel),National Average,739.516667,0.720150,2015-11-15,2016-10-15,12
"Fuel (kerosene, paraffin)",National Average,591.325000,0.575825,2015-11-15,2016-10-15,12
Fuel (petrol),National Average,752.558333,0.732867,2015-11-15,2016-10-15,12


,,price_avg,usdprice_avg,start_date,end_date,date_count
commodity,market,,,,,
Fuel (diesel),National Average,667.30,0.64985,2013-05-15,2013-06-15,2
"Fuel (kerosene, paraffin)",National Average,565.25,0.55045,2013-05-15,2013-06-15,2
Fuel (petrol),National Average,687.55,0.66955,2013-05-15,2013-06-15,2


,,price_avg,usdprice_avg,start_date,end_date,date_count
commodity,market,,,,,
Fuel (diesel),National Average,678.000,0.66025,2013-02-15,2013-06-15,4
"Fuel (kerosene, paraffin)",National Average,583.950,0.56865,2013-02-15,2013-06-15,4
Fuel (petrol),National Average,698.575,0.68030,2013-02-15,2013-06-15,4


,,price_avg,usdprice_avg,start_date,end_date,date_count
commodity,market,,,,,
Fuel (diesel),National Average,593.9875,0.578437,2012-07-15,2013-06-15,8
"Fuel (kerosene, paraffin)",National Average,504.6250,0.491400,2012-07-15,2013-06-15,8
Fuel (petrol),National Average,608.2875,0.592363,2012-07-15,2013-06-15,8
